<a href="https://colab.research.google.com/github/sahilaf/Test_models/blob/main/Preprocess_gstalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Make directories

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p Obama_data/{ori_imgs,parsing,torso_imgs}

# Convert Original frames from the original video

In [ ]:
!ffmpeg -i my_video.mov -qscale:v 2 Obama_data/ori_imgs/%05d.jpg

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [2]:
!mv /content/drive/MyDrive/Obama_data /content/

In [ ]:
!mv /content/output/my_video.csv /content/Obama_data/

In [126]:
!cp -r /content/AD-NeRF/ /content/drive/MyDrive/

In [ ]:
!apt-get install imagemagick -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [ ]:
!mogrify -format jpg Obama_data/gt_imgs/*.bmp

In [ ]:
!ls

au.csv	gt_imgs  Obama_data.mp4  ori_imgs  parsing  torso_imgs


In [ ]:
!rm ori_imgs/*.jpg

In [ ]:
%cd Obama_data

/content/Obama_data


In [ ]:
!ls

au.csv	gt_imgs  Obama_data.mp4  ori_imgs  parsing  torso_imgs


In [ ]:
import pandas as pd, numpy as np, os, json

csv = pd.read_csv('au.csv')
out_dir = 'ori_imgs'
os.makedirs(out_dir, exist_ok=True)

for i, row in csv.iterrows():
    lms = np.array([[row[f'x_{j}'], row[f'y_{j}']] for j in range(68)])
    np.savetxt(os.path.join(out_dir, f"{i}.lms"), lms)


In [ ]:
import os

folder = "/content/Obama_data/ori_imgs"  # change if your path differs
files = sorted(os.listdir(folder))  # will correctly sort 00001.jpg, 00002.jpg, etc.

# filter only image files
files = [f for f in files if f.lower().endswith('.jpg')]

for i, filename in enumerate(files):
    src = os.path.join(folder, filename)
    dst = os.path.join(folder, f"{i}.jpg")
    os.rename(src, dst)

print(f"✅ Renamed {len(files)} files from 0.jpg to {len(files)-1}.jpg")


✅ Renamed 892 files from 0.jpg to 891.jpg


In [ ]:
import os
import re

folder = "/content/drive/MyDrive/Obama_data/gt_imgs"

# Function to extract number from filename
def get_number(filename):
    nums = re.findall(r'\d+', filename)
    return int(nums[0]) if nums else -1

# Get only .jpg files and sort them numerically
files = sorted(
    [f for f in os.listdir(folder) if f.lower().endswith('.jpg')],
    key=get_number
)

# Rename sequentially
for i, filename in enumerate(files):
    src = os.path.join(folder, filename)
    dst = os.path.join(folder, f"{i}.jpg")
    os.rename(src, dst)

print(f"✅ Renamed {len(files)} files numerically from 0.jpg to {len(files)-1}.jpg")


✅ Renamed 892 files numerically from 0.jpg to 891.jpg


# Parsing

In [9]:
!git clone https://github.com/YudongGuo/AD-NeRF.git
%cd AD-NeRF

Cloning into 'AD-NeRF'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 161 (delta 21), reused 19 (delta 19), pack-reused 133 (from 1)
Receiving objects: 100% (161/161), 156.66 MiB | 14.44 MiB/s, done.
Resolving deltas: 100% (60/60), done.
/content/AD-NeRF


In [10]:
!pip install imageio[ffmpeg] scipy opencv-python numpy tqdm

In [11]:
!mkdir -p data_utils/face_parsing
!wget "https://github.com/YudongGuo/AD-NeRF/raw/master/data_util/face_parsing/79999_iter.pth" -O data_utils/face_parsing/79999_iter.pth


--2025-10-12 10:38:02--  https://github.com/YudongGuo/AD-NeRF/raw/master/data_util/face_parsing/79999_iter.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/YudongGuo/AD-NeRF/master/data_util/face_parsing/79999_iter.pth [following]
--2025-10-12 10:38:02--  https://raw.githubusercontent.com/YudongGuo/AD-NeRF/master/data_util/face_parsing/79999_iter.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53289463 (51M) [application/octet-stream]
Saving to: ‘data_utils/face_parsing/79999_iter.pth’

data_utils/face_par 100%[===================>]  50.82M  --.-KB/s    in 0.1s    

2025-10-12 10:38:0

In [17]:
!mkdir -p data_util/face_tracking/3DMM
!wget https://github.com/wuhaozhe/AD-NeRF/raw/main/data_util/face_tracking/3DMM/BFM_model_front.mat -O data_util/face_tracking/3DMM/01_MorphableModel.mat


--2025-10-12 10:54:22--  https://github.com/wuhaozhe/AD-NeRF/raw/main/data_util/face_tracking/3DMM/BFM_model_front.mat
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-10-12 10:54:22 ERROR 404: Not Found.



In [24]:
!ls

dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [26]:
!ls -lh /content/AD-NeRF/data_util/face_tracking/3DMM/01_MorphableModel.mat


-rw-r--r-- 1 root root 11M Oct 12 11:16 /content/AD-NeRF/data_util/face_tracking/3DMM/01_MorphableModel.mat


In [53]:
from scipy.io import loadmat

path = '/content/01_MorphableModel_fixed_v73.mat'
model = loadmat(path)

print(model.keys())


NotImplementedError: Please use HDF reader for matlab v7.3 files, e.g. h5py

In [75]:
!ls

dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [74]:
%cd AD-NeRF/

/content/AD-NeRF


In [76]:
%cd data_util/face_tracking
!python convert_BFM.py
%cd ../../


/content/AD-NeRF/data_util/face_tracking
3DMM info saved successfully!
/content/AD-NeRF


In [52]:
import os

path = '/content/01_MorphableModel_fixed_v73.mat'

# Check if file exists and size
if os.path.exists(path):
    file_size = os.path.getsize(path)
    print(f"File size: {file_size} bytes")

    if file_size == 0:
        print("ERROR: File is empty!")
    else:
        print("File exists and has content")
else:
    print("ERROR: File not found!")

File size: 20971520 bytes
File exists and has content


In [51]:
with open('/content/01_MorphableModel_fixed_v73.mat','rb') as f:
    header = f.read(4)
print(header)


b'MATL'


In [49]:
import os
from scipy.io import loadmat, whosmat

path = '/content/01_MorphableModel_fixed_v73.mat'

# Check file size and basic integrity
file_size = os.path.getsize(path)
print(f"File size: {file_size} bytes")

# Try to inspect file contents without fully loading
try:
    file_info = whosmat(path)
    print("Variables in file:", file_info)
except Exception as e:
    print(f"Could not inspect file: {e}")

File size: 16777216 bytes
Could not inspect file: Please use HDF reader for matlab v7.3 files, e.g. h5py


In [54]:
# Try different byte order specifications
byte_orders = ['native', 'little', 'big', '=']

for byte_order in byte_orders:
    try:
        model = loadmat(path, byte_order=byte_order)
        print(f"Success with byte_order='{byte_order}'")
        print("Keys found:", list(model.keys()))
        break
    except Exception as e:
        print(f"Failed with byte_order='{byte_order}': {e}")

Failed with byte_order='native': Please use HDF reader for matlab v7.3 files, e.g. h5py
Failed with byte_order='little': Please use HDF reader for matlab v7.3 files, e.g. h5py
Failed with byte_order='big': Please use HDF reader for matlab v7.3 files, e.g. h5py
Failed with byte_order='=': Please use HDF reader for matlab v7.3 files, e.g. h5py


In [58]:
!ls


dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [59]:
%cd data_util/face_tracking/3DMM/

/content/AD-NeRF/data_util/face_tracking/3DMM


In [68]:
!ls -lh 01_MorphableModel_fixed_v73.mat


-rw-r--r-- 1 root root 924M Oct 12 13:19 01_MorphableModel_fixed_v73.mat


In [69]:
import h5py
with h5py.File('01_MorphableModel_fixed_v73.mat', 'r') as f:
    print(list(f.keys()))


['#refs#', 'ans', 'data', 'data1', 'fileInfo', 'info', 'loadedData', 'segMB', 'segMM', 'segbin', 'shapeEV', 'shapeMU', 'shapePC', 'texEV', 'texMU', 'texPC', 'tl']


In [80]:
!ls

dataset    environment.yml  NeRFs	pretrained_models  README.md
data_util  LICENSE	    paper_data	process_data.sh


In [85]:
!pip install face-alignment


In [86]:
!python data_util/process_data.py /content/my_video.mov

usage: process_data.py [-h] [--id ID] [--step STEP]
process_data.py: error: unrecognized arguments: /content/my_video.mov


In [93]:
!pip install resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.0 MB/s eta 0:00:00


In [119]:
!pip install pytorch3d

ERROR: Could not find a version that satisfies the requirement pytorch3d (from versions: none)
ERROR: No matching distribution found for pytorch3d


In [120]:
import torch
print(torch.__version__)
print(torch.version.cuda)


2.8.0+cu126
12.6


In [ ]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-p3ywdojn
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-p3ywdojn
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit fc6a6b895185a502c4064fc5ea7ecb760fd1a0d6
  Preparing metadata (setup.py) ... done
  Using cached iopath-0.1.10.tar.gz (42 kB)
  Preparing metadata (setup.py) ... done
  Using cached portalocker-3.2.0-py3-none-any.whl.metadata (8.7 kB)
Using cached portalocker-3.2.0-py3-none-any.whl (22 kB)


In [95]:
!pip install python_speech_features

  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=fc4fcb728c925cbd8362c674559ba9a8144b664137608d900ddbfa104c0ab357
  Stored in directory: /root/.cache/pip/wheels/60/90/3c/4b5996a95d363fa14525597a19146a940bec467b44b2a14580
Successfully built python_speech_features


In [113]:
!pip install configargparse

In [104]:
!python data_util/process_data.py --id Obama --step 0

--- Step0: extract deepspeech feature ---
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab

In [105]:
!python data_util/process_data.py --id Obama --step 1

--- Step1: extract images from vids ---


In [107]:
!python data_util/process_data.py --id Obama --step 2

--- Step 2: detect landmarks ---
Downloading: "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" to /root/.cache/torch/hub/checkpoints/s3fd-619a316812.pth
100% 85.7M/85.7M [00:06<00:00, 13.5MB/s]
Downloading: "https://www.adrianbulat.com/downloads/python-fan/2DFAN4-cd938726ad.zip" to /root/.cache/torch/hub/checkpoints/2DFAN4-cd938726ad.zip
100% 91.9M/91.9M [00:09<00:00, 10.5MB/s]


In [114]:
!python data_util/process_data.py --id Obama --step 3

--- Step 3: face parsing ---
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 102MB/s] 
processed parsing 100
processed parsing 200
processed parsing 300
processed parsing 400
processed parsing 500
processed parsing 600
processed parsing 700
processed parsing 800
processed parsing 900
processed parsing 1000
processed parsing 1100
processed parsing 1200
processed parsing 1300
processed parsing 1400
processed parsing 1500
processed parsing 1600
processed parsing 1700
processed parsing 1800
processed parsing 1900
processed parsing 2000
processed parsing 2100
processed parsing 2200
processed parsing 2300
processed parsing 2400
processed parsing 2500
processed parsing 2600
processed parsing 2700
processed parsing 2800
processed parsing 2900
processed parsing 3000
processed parsing 3100
processed parsing 3200
processed parsing 3300
processed parsing 3400
processed parsing 3500
p

In [115]:
!python data_util/process_data.py --id Obama --step 4

--- Step 4: extract background image ---
(400, 202500, 1)
(132581, 2)
449 0


In [116]:
!python data_util/process_data.py --id Obama --step 5

--- Step 5: save training images ---


In [123]:
!python data_util/process_data.py --id Obama --step 6

--- Estimate Head Pose ---
Traceback (most recent call last):
  File "/content/AD-NeRF/data_util/face_tracking/face_tracker.py", line 11, in <module>
    from render_3dmm import Render_3DMM
  File "/content/AD-NeRF/data_util/face_tracking/render_3dmm.py", line 5, in <module>
    from pytorch3d.structures import Meshes
ModuleNotFoundError: No module named 'pytorch3d'


In [112]:
!python data_util/process_data.py --id Obama --step 7

--- Step 7: Save Transform Param ---
Traceback (most recent call last):
  File "/content/AD-NeRF/data_util/process_data.py", line 193, in <module>
    params_dict = torch.load(os.path.join(id_dir, 'track_params.pt'))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1484, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 759, in _open_file_like
    return _open_file(name_or_buffer, mode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 740, in __init__
    super().__init__(open(name, mode))
                     ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'dataset/Obama/track_params.pt'
